<a href="https://colab.research.google.com/github/Parikshit07111996/School-Of-AI/blob/master/EVA4_Session_4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3)  # I : 28, o 26, ch 32 #nn.Conv2d(in_channels, out_channels, kernel_size)
        self.d1 =nn.Dropout(0.1)
        self.bn1 = nn.BatchNorm2d(8)

        self.conv2 = nn.Conv2d(8, 16, 3) # i 26, o 24, ch 16
        self.d2 =nn.Dropout(0.1)
        self.bn2 = nn.BatchNorm2d(16)

        self.pool1 = nn.MaxPool2d(2, 2) # i 24, o 12, ch 16
       
        self.conv3 = nn.Conv2d(16, 16, 3) # i 12, o 10, ch 16
        self.d3 =nn.Dropout(0.1)
        self.bn3 = nn.BatchNorm2d(16)
        
        self.conv4 = nn.Conv2d(16, 32, 3) # i 10 , o 8, ch 32
        self.d4 =nn.Dropout(0.1)
        self.bn4 = nn.BatchNorm2d(32)

        self.pool2 = nn.MaxPool2d(2, 2) # i 8, o 4, ch 64

        self.conv5 = nn.Conv2d(32, 32, 3) # i 4, o 2, ch 64 
        self.d5 =nn.Dropout(0.1)
        self.bn5 = nn.BatchNorm2d(32)
        
        self.conv6 = nn.Conv2d(32, 10, 2) # i 2, o 1, ch  10


    def forward(self, x):
      x = self.conv1(x)
      x = F.relu(x)
      x = self.d1(x)
      x = self.bn1(x)

      x = self.conv2(x)
      x = F.relu(x)
      x = self.d2(x)
      x = self.bn2(x)

      x = self.pool1(x)
     
      x = self.conv3(x)
      x = F.relu(x)
      x = self.d3(x)
      x = self.bn3(x)

      x = self.conv4(x)
      x = F.relu(x)
      x = self.d4(x)
      x = self.bn4(x)

      x = self.pool2(x)
      
      x = self.conv5(x)
      x = F.relu(x)
      x = self.d5(x)
      x = self.bn5(x)

      x = self.conv6(x)
      x = F.relu(x)

      x = x.view(-1, 10)
      return F.log_softmax(x)


In [47]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
           Dropout-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4           [-1, 16, 24, 24]           1,168
           Dropout-5           [-1, 16, 24, 24]               0
       BatchNorm2d-6           [-1, 16, 24, 24]              32
         MaxPool2d-7           [-1, 16, 12, 12]               0
            Conv2d-8           [-1, 16, 10, 10]           2,320
           Dropout-9           [-1, 16, 10, 10]               0
      BatchNorm2d-10           [-1, 16, 10, 10]              32
           Conv2d-11             [-1, 32, 8, 8]           4,640
          Dropout-12             [-1, 32, 8, 8]               0
      BatchNorm2d-13             [-1, 32, 8, 8]              64
        MaxPool2d-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128 #### Batch Sixe = No:Of Images we are looking a simultaneously 


# Train loader is a way to ingest our images and send to our model
# First we are converting into Tensor as we want the model to run on GPU
#Second we are normalizing the data set (mean , std)
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [50]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0569, Accuracy: 9842/10000 (98.420%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0478, Accuracy: 9857/10000 (98.570%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0196, Accuracy: 9943/10000 (99.430%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0207, Accuracy: 9935/10000 (99.350%)



loss=0.006801530718803406 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 42.22it/s]



Test set: Average loss: 0.0207, Accuracy: 9937/10000 (99.370%)

